<h1> Connection Serial <h1>

In [56]:
%serialconnect to --port=/dev/ttyUSB0 --baud=115200

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


<h1> Setup Network </h1>

In [57]:
# All imports 
import micropython

# print(micropython.mem_info('verbose'))



import usocket as socket
from machine import Pin
import network
import esp
import utime
import ujson
import gc
import socket
import uhashlib
import ubinascii
import uos


esp.osdebug(None)
gc.collect()

sta_if = network.WLAN(network.STA_IF)
ap_if = network.WLAN(network.AP_IF)
print('Station Up: ', sta_if.active())
print('Acess Point Up: ', ap_if.active())


def connectX(typeX):
    typeX.active(True)
    typeX.config(dhcp_hostname='PrimeTime_uPython'),
    print('Station Up: ', sta_if.active())
    print('Acess Point Up: ', ap_if.active())


def scanX():
    print('Status: %s\n' % sta_if.status())
    station_default = False
    stations_list = sta_if.scan()
#     print('\ntest1: ', stations_list)
    for i, n in enumerate(stations_list, start=0):
        print('i= {0} n= {1}'.format(i, n))
#         if n[0] == b'SM-A1kerr' or n[0] == b'Atalyah':
#             print('\nFOUND SID!: ', n[0])
#             station_default = True

    if station_default is False:
        print('\n*** PLEASE ENABLE A DEFAULT STATION! ***')
    return stations_list


# Make a dictionary out of a byte Object b''
def get_meta(c):
    c = c.decode("utf-8")
    holdX = c.split('\r\n')
#     print(type(holdX))
    holdy = {}
    for l in holdX:
        dataX = len(l.split(':'))
        if dataX > 1:
            holdy[str(l.split(':')[0])] = str(l.split(':')[1])
        else:
            holdy[str(l.split(':')[0])] = str(l.split(':')[0])

    return holdy

print('OS Information = ',  uos.uname())

Station Up:  False
Acess Point Up:  False
OS Information =  (sysname='esp32', nodename='esp32', release='1.12.0', version='v1.12-164-g7679e3be9 on 2020-02-12', machine='ESP32 module (spiram) with ESP32')


In [58]:
connectX(sta_if)
if sta_if.active():
    stations_list = scanX()
elif ap_if.active():
    print('Acess Point Up: ', ap_if.active())

print(sta_if.config('essid'))
# print(sta_if.config('channel'))

Station Up:  True
Acess Point Up:  False
Status: 1000

i= 0 n= (b'ASUS2_4_PRIMETIME_905', b'\x08`n\xbc\x02\x08', 6, -42, 3, False)
i= 1 n= (b'Atalyah', b'\x08`n\xbc\x02\t', 6, -43, 3, False)
i= 2 n= (b'dlink_2_4GHz_PrimeTime', b'@\x9b\xcd\xa0ih', 11, -50, 4, False)
i= 3 n= (b'VIRGIN463', b'\xa8\x9a\x93\x1d\xf8\x9e', 1, -55, 3, False)
i= 4 n= (b'BELL366', b'\x90r\x82\xfe\x8e&', 6, -68, 3, False)
i= 5 n= (b'Family1', b'B\xf2\x01\xf8\x94W', 11, -83, 3, False)

*** PLEASE ENABLE A DEFAULT STATION! ***



In [59]:
selected_SID = stations_list[1][0]
pw = '9054830760'

In [60]:
# connectX(sta_if)
# stations_list = []



# print('Connecting to: {0}'.format(stations_list[0][0]))
while not (sta_if.isconnected()):
    print('Trying to connet to server!')
#     if 'Atalyah' in stations_list : 
#     print(stations_list[0][0])
    for x in stations_list:
#         print(x)
#         if b'Atalyah' in x or b'SM-A1kerr' in x:
        if selected_SID in x or b'SM-A1kerr' in x:
#             print(x[0])
            sta_if.connect(x[0], pw)
    utime.sleep(1)


print('Status: {0}\n'.format(sta_if.status(), sta_if.isconnected()),
      ('\nIs Connected Status: {0}'.format(sta_if.isconnected())),
      ('\nConnected To WIFI SID: {0}'.format(sta_if.config('essid'))),
      ('\nConnected Hostname: {0}'.format(sta_if.config('dhcp_hostname'))),
      ('\nifconfig Status: {0}'.format(sta_if.ifconfig()))
      )

# print(micropython.mem_info('verbose'))

Trying to connet to server!
Trying to connet to server!
Status: 1010
 
Is Connected Status: True 
Connected To WIFI SID: Atalyah 
Connected Hostname: PrimeTime_uPython 
ifconfig Status: ('192.168.1.240', '255.255.255.0', '192.168.1.1', '192.168.1.1')


<h1>Get webpage </h1>

In [66]:
def main(use_stream=False, www_page='google.com'):
    end_line = b'\r\n\r\n'
    s = socket.socket()
    
    ai = socket.getaddrinfo(www_page, 80)
    print('Address Info: ', ai)
    addr = ai[0][-1]
    
    print('Connection Address: ', addr)
    s.connect(addr)
    
    if use_stream:
        print('Connecting Using Streaming\n')
        s = s.makefile('rwb', 0)
        s.write(b'GET / HTTP/1.0'+end_line)
        while True:
            chunk =  s.read(1024)
            if len(chunk) == 0:
                break
            print('\n\n', chunk)
    else:
        print('Connecting Using 4096 byte buffer\n')
        s.send(b'GET / HTTP/1.0'+end_line)
        print(b'GET / HTTP/1.0'+end_line)
        print('\n', s.recv(4096))
        
#     print(micropython.mem_info('verbose'))
    s.close()
    
    
main(use_stream=True, www_page='yahoo.com')

Address Info:  [(2, 1, 0, 'yahoo.com', ('98.137.246.7', 80))]
Connection Address:  ('98.137.246.7', 80)
Connecting Using Streaming



 b'HTTP/1.0 400 Host Header Required\r\nDate: Sun, 15 Mar 2020 01:57:25 GMT\r\nServer: ATS\r\nCache-Control: no-store\r\nContent-Type: text/html\r\nContent-Language: en\r\nX-Frame-Options: SAMEORIGIN\r\nContent-Length: 4383\r\n\r\n<!DOCTYPE html>\n<html lang="en-us">\n  <head>\n    <meta http-equiv="content-type" content="text/html; charset=UTF-8">\n    <meta charset="utf-8">\n    <title>Yahoo</title>\n    <meta name="viewport" content="width=device-width,initial-scale=1,minimal-ui">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n    <style>\n      html {\n          height: 100%;\n      }\n      body {\n          background: #fafafc url(https://s.yimg.com/nn/img/sad-panda-201402200631.png) 50% 50%;\n          background-size: cover;\n          height: 100%;\n          text-align: center;\n          font: 300 18px "helvetica neue", h

<br>
<h1>Websocket Return Header Format</h1>
<p>
HTTP/1.1 101 Switching Protocols<br>
Upgrade: websocket<br>
Connection: Upgrade<br>
Sec-WebSocket-Accept: s3pPLMBiTxaQ9kYGzzhZRbK+xOo=<br>
(remember each header line ends with \r\n and put an extra \r\n after the last one to indicate the end of the header)
</p>
<p>
-hashed key- The server takes the value of Sec-WebSocket-Key sent in handshake request and then appends Globally Unique Identifier (GUID, [RFC4122]) “258EAFA5-E914-47DA-95CA-C5AB0DC85B11” takes SHA-1 of the new value, and is then base64 encoded. This is unlikely to be used by network endpoints that do not understand the WebSocket Protocol.
</p>

In [10]:
print(socket.getaddrinfo('', 80))

[(2, 1, 0, '0.0.0.0', ('0.0.0.0', 80))]


In [ ]:
# Complete project details at https://RandomNerdTutorials.com


addr_socket = socket.getaddrinfo('', 80)[0][4]
# addr_socket = socket.getaddrinfo('192.168.43.34', 80)[0][4]
# addr = socket.getaddrinfo('192.168.43.34', 80)
led = Pin(2, Pin.OUT)
key_tag = 'Sec-WebSocket-Key'
guid = b'258EAFA5-E914-47DA-95CA-C5AB0DC85B11'


def web_page():
    if led.value() == 1:
        gpio_state = "ON"
    else:
        gpio_state = "OFF"

    html = """<html><head> <title>ESP Web Server</title> <meta name = "viewport"
    content="width=device-width, initial-scale=1">
    <link rel="icon" href="data:,"> <style>html{font-family: Helvetica;
    display:inline-block; margin: 0px auto; text-align: center;}
    h1{color: #0F3376; padding: 2vh;}p{font-size: 1.5rem;}.button{display:
    inline-block; background-color: #e7bd3b; border: none;
    border-radius: 4px; color: white; padding: 16px 40px; text-decoration:
    none; font-size: 30px; margin: 2px; cursor: pointer;}
    .button2{background-color: #4286f4;}</style></head><body>
    <h1>ESP Web Server</h1>
    <p>GPIO state: <strong>""" + gpio_state + """</strong></p><p>
    <a href="/?led=on"><button class="button">ON</button></a></p>
    <p><a href="/?led=off"><button class="button button2">OFF</button>
    </a></p></body></html>"""
    return html


html2 = """<!DOCTYPE html>
<html>
<head> <title>ESP8266 Pins</title> </head>
<body> <h1>ESP8266 Pins</h1>
    <table border="1"> <tr><th>Pin</th><th>Value</th></tr> %s </table>
</body>
</html>
"""

s=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('', 80))
print('Bind: ', addr_socket)
# print('Bind: %s' % addr[0])
# print('Bind: %s' % addr[0][-1])
s.listen(5)

while True:
    conn, addr = s.accept()
    print('Got a connection!: ')
    print('Got a connection from %s' % str(addr))
    request=conn.recv(1024)
#     conn.send(html2)
#     request = request.decode()
    request_meta = get_meta(request)
    print('Content = %s' % request_meta)
    if key_tag in request_meta:
        print('Sec-Weboscket-Key: ', request_meta[key_tag])
        key_hash = uhashlib.sha1(guid)
        key_hash.update(request_meta[key_tag])
#         key_hash = uhashlib.sha1(request_meta[key_tag])
#         key_hash.update(guid)
        key_sha1 = key_hash.digest()
        key_sha1 = ubinascii.b2a_base64(key_sha1)
        header1 = 'HTTP/1.1 101 Switching Protocols\r\n'
        header2 = 'Upgrade: websocket\r\n'
        header3 = 'Connection: Upgrade\r\n'
        header_sec = 'Sec-WebSocket-Accept: {0}'.format(key_sha1)
        header_end = '\r\n\r\n'
        socketX_upgrade = bytes(header1+header2+header3+header_sec+header_end, 'utf8')
        print('SHA1 Key: ', key_sha1)
        conn.send(socketX_upgrade)
    else:
        print('No Key Found')
          
    
    conn.close()

print('Server closed: ')

Bind:  ('0.0.0.0', 80)
...................................................................................................................Got a connection!: 
Got a connection from ('192.168.1.106', 38541)
Content = {'User-Agent': ' Mozilla/5.0 (Linux; Android 8.0.0; SM-G930W8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Mobile Safari/537.36', 'Connection': ' keep-alive', 'Accept-Language': ' en-GB,en-US;q=0.9,en;q=0.8,da;q=0.7', 'Accept': ' text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', '': '', 'GET / HTTP/1.1': 'GET / HTTP/1.1', 'Accept-Encoding': ' gzip, deflate', 'Host': ' 192.168.1.240', 'Upgrade-Insecure-Requests': ' 1'}
No Key Found
Got a connection!: 
Got a connection from ('192.168.1.106', 38542)
...Content = {'': ''}
No Key Found
................................................................

<br>
<p>
Format of getaddrinfo:  [(2, 1, 0, '192.168.43.34', ('192.168.43.34', 80))]
</p>